### Metrics calculation and some plots
Use this notebook to calculate the BWT and FWT scores for the images the network. This works for the registration predictions as well as U-Net predictions. Just set the paths to the networks predictions correctly and the notebook will do the rest.

#### Set paths

In [1]:
sam_exps = '/local/scratch/aranem/MIDL_2024/MIDL_2024_trained_models/Seq'
reh_exps =   '/local/scratch/aranem/MIDL_2024/MIDL_2024_trained_models/Reh'
ewc_exps =   '/local/scratch/aranem/MIDL_2024/MIDL_2024_trained_models/EWC'
rwalk_exps = '/local/scratch/aranem/MIDL_2024/MIDL_2024_trained_models/RWalk'

#### Import necessary libraries

In [2]:
from math import pi
import numpy as np
import pandas as pd
import seaborn as sns
from time import sleep
import SimpleITK as sitk
import matplotlib.pyplot as plt
from torch.autograd import Variable
import os, copy, monai, torch
from tqdm.notebook import trange, tqdm

#### Helpful functions and other stuffs

In [3]:
def adjust_lightness(color, amount=0.5):
    import matplotlib.colors as mc
    import colorsys
    try:
        c = mc.cnames[color]
    except:
        c = color
    c = colorsys.rgb_to_hls(*mc.to_rgb(c))
    return colorsys.hls_to_rgb(c[0], max(0, min(1, amount * c[1])), c[2])
    
def trunc(values, decs=0):
    return np.trunc(values*10**decs)/(10**decs)

### SAM results

In [4]:
models = [x for x in os.listdir(sam_exps) if 'sam_adapter' in x]

models.sort()
dices_s = dict()
for model in models:
    dices_s[model.replace('_100_lr-1e-4-with-adapter-lr-1e-3', '').replace('sam_adapter_', '')] = dict()
    preds_ = [x for x in os.listdir(os.path.join(sam_exps, model)) if 'inference' in x]

    for pred in preds_:
        preds = os.path.join(sam_exps, model, pred)
        print(f"Looking in {preds}..")
        # -- Load the data -- #
        try:
            ds = [x for x in os.listdir(preds) if 'csv' not in x]
            ds.sort()
            print(f'Dice for model {model}:')
        except:
            continue
        
        for datas in ds:
            dices_ = list()
            dices_s[model.replace('_100_lr-1e-4-with-adapter-lr-1e-3', '').replace('sam_adapter_', '')][datas.split('_')[0].replace('Task', '')] = dict()
            cases = [x for x in os.listdir(os.path.join(preds, datas)) if 'DS_' not in x]
            for case in cases:
                dices_s[model.replace('_100_lr-1e-4-with-adapter-lr-1e-3', '').replace('sam_adapter_', '')][datas.split('_')[0].replace('Task', '')][case] = dict()
                gt = np.int32(sitk.GetArrayFromImage(sitk.ReadImage(os.path.join(preds, datas, case, 'seg_gt.nii.gz'))))
                y_p = sitk.GetArrayFromImage(sitk.ReadImage(os.path.join(preds, datas, case, 'pred_seg.nii.gz')))
                y_p_t = torch.from_numpy(y_p).to(0).unsqueeze(0).unsqueeze(0).float()#.permute(0, 4, 1, 2, 3)
                gt_t = torch.from_numpy(gt).to(0).unsqueeze(0).unsqueeze(0).float()#.permute(0, 4, 1, 2, 3)
                dice = monai.metrics.DiceMetric(include_background=False, ignore_empty=False)(y_p_t, gt_t).cpu().item()
                dices_s[model.replace('_100_lr-1e-4-with-adapter-lr-1e-3', '').replace('sam_adapter_', '')][datas.split('_')[0].replace('Task', '')][case]['pred'] = np.round(dice, 4)
                dices_.append(dice)

            print(f"{datas}: Mean dice +/- std: {trunc(np.round(np.mean(dices_), decimals=4)*100, 2)} ({trunc(np.round(np.std(dices_), decimals=4)*100, 2)})") 
            
            dices_s[model.replace('_100_lr-1e-4-with-adapter-lr-1e-3', '').replace('sam_adapter_', '')][datas.split('_')[0].replace('Task', '')]['mean_dice_moved'] = np.mean(dices_)
            dices_s[model.replace('_100_lr-1e-4-with-adapter-lr-1e-3', '').replace('sam_adapter_', '')][datas.split('_')[0].replace('Task', '')]['mean_std_moved'] = np.std(dices_)
            dices_s[model.replace('_100_lr-1e-4-with-adapter-lr-1e-3', '').replace('sam_adapter_', '')][datas.split('_')[0].replace('Task', '')]['mean_dice_std_moved'] = str(np.round(np.mean(dices_), 4)) + '(' + str(np.round(np.std(dices_), 4)) +')'
        print()

Looking in /local/scratch/aranem/MIDL_2024/MIDL_2024_trained_models/Seq/sam_adapter_99_100_lr-1e-4-with-adapter-lr-1e-3/inference..
Dice for model sam_adapter_99_100_lr-1e-4-with-adapter-lr-1e-3:
Task97_DecathHip: Mean dice +/- std: 30.04 (8.15)
Task98_Dryad: Mean dice +/- std: 34.82 (7.06)
Task99_HarP: Mean dice +/- std: 39.54 (14.18)



In [ ]:
# Rehearsal
models = [x for x in os.listdir(reh_exps) if 'sam_adapter' in x and 'joint' not in x]

models.sort()
dices_reh = dict()
for model in models:
    dices_reh[model.replace('_100_lr-1e-4-with-adapter-lr-1e-3', '').replace('sam_adapter_', '')] = dict()
    preds_ = [x for x in os.listdir(os.path.join(reh_exps, model)) if 'inference' in x]
    for pred in preds_:
        preds = os.path.join(reh_exps, model, pred)
        print(f"Looking in {preds}..")
        # -- Load the data -- #
        try:
            ds = [x for x in os.listdir(preds) if 'csv' not in x]
            ds.sort()
            print(f'Dice for model {model}:')
        except:
            continue
        
        for datas in ds:
            dices_ = list()
            dices_reh[model.replace('_100_lr-1e-4-with-adapter-lr-1e-3', '').replace('sam_adapter_', '')][datas.split('_')[0].replace('Task', '')] = dict()
            cases = [x for x in os.listdir(os.path.join(preds, datas)) if 'DS_' not in x]
            for case in cases:
                dices_reh[model.replace('_100_lr-1e-4-with-adapter-lr-1e-3', '').replace('sam_adapter_', '')][datas.split('_')[0].replace('Task', '')][case] = dict()
                gt = np.int32(sitk.GetArrayFromImage(sitk.ReadImage(os.path.join(preds, datas, case, 'seg_gt.nii.gz'))))
                y_p = sitk.GetArrayFromImage(sitk.ReadImage(os.path.join(preds, datas, case, 'pred_seg.nii.gz')))
                y_p_t = torch.from_numpy(y_p).to(0).unsqueeze(0).unsqueeze(0).float()#.permute(0, 4, 1, 2, 3)
                gt_t = torch.from_numpy(gt).to(0).unsqueeze(0).unsqueeze(0).float()#.permute(0, 4, 1, 2, 3)
                dice = monai.metrics.DiceMetric(include_background=False, ignore_empty=False)(y_p_t, gt_t).cpu().item()
                dices_reh[model.replace('_100_lr-1e-4-with-adapter-lr-1e-3', '').replace('sam_adapter_', '')][datas.split('_')[0].replace('Task', '')][case]['pred'] = np.round(dice, 4)
                dices_.append(dice)

            print(f"{datas}: Mean dice +/- std: {trunc(np.round(np.mean(dices_), decimals=4)*100, 2)} ({trunc(np.round(np.std(dices_), decimals=4)*100, 2)})") 
            
            dices_reh[model.replace('_100_lr-1e-4-with-adapter-lr-1e-3', '').replace('sam_adapter_', '')][datas.split('_')[0].replace('Task', '')]['mean_dice_moved'] = np.mean(dices_)
            dices_reh[model.replace('_100_lr-1e-4-with-adapter-lr-1e-3', '').replace('sam_adapter_', '')][datas.split('_')[0].replace('Task', '')]['mean_std_moved'] = np.std(dices_)
            dices_reh[model.replace('_100_lr-1e-4-with-adapter-lr-1e-3', '').replace('sam_adapter_', '')][datas.split('_')[0].replace('Task', '')]['mean_dice_std_moved'] = str(np.round(np.mean(dices_), 4)) + '(' + str(np.round(np.std(dices_), 4)) +')'
        print()

In [ ]:
# EWC
models = [x for x in os.listdir(ewc_exps) if 'sam_adapter' in x and 'joint' not in x]

models.sort()
dices_ewc = dict()
for model in models:
    dices_ewc[model.replace('_100_lr-1e-4-with-adapter-lr-1e-3', '').replace('sam_adapter_', '')] = dict()
    preds_ = [x for x in os.listdir(os.path.join(ewc_exps, model)) if 'inference' in x]
    for pred in preds_:
        preds = os.path.join(ewc_exps, model, pred)
        print(f"Looking in {preds}..")
        # -- Load the data -- #
        try:
            ds = [x for x in os.listdir(preds) if 'csv' not in x]
            ds.sort()
            print(f'Dice for model {model}:')
        except:
            continue
        
        for datas in ds:
            dices_ = list()
            dices_ewc[model.replace('_100_lr-1e-4-with-adapter-lr-1e-3', '').replace('sam_adapter_', '')][datas.split('_')[0].replace('Task', '')] = dict()
            cases = [x for x in os.listdir(os.path.join(preds, datas)) if 'DS_' not in x]
            for case in cases:
                dices_ewc[model.replace('_100_lr-1e-4-with-adapter-lr-1e-3', '').replace('sam_adapter_', '')][datas.split('_')[0].replace('Task', '')][case] = dict()
                gt = np.int32(sitk.GetArrayFromImage(sitk.ReadImage(os.path.join(preds, datas, case, 'seg_gt.nii.gz'))))
                y_p = sitk.GetArrayFromImage(sitk.ReadImage(os.path.join(preds, datas, case, 'pred_seg.nii.gz')))
                y_p_t = torch.from_numpy(y_p).to(0).unsqueeze(0).unsqueeze(0).float()#.permute(0, 4, 1, 2, 3)
                gt_t = torch.from_numpy(gt).to(0).unsqueeze(0).unsqueeze(0).float()#.permute(0, 4, 1, 2, 3)
                dice = monai.metrics.DiceMetric(include_background=False, ignore_empty=False)(y_p_t, gt_t).cpu().item()
                dices_ewc[model.replace('_100_lr-1e-4-with-adapter-lr-1e-3', '').replace('sam_adapter_', '')][datas.split('_')[0].replace('Task', '')][case]['pred'] = np.round(dice, 4)
                dices_.append(dice)

            print(f"{datas}: Mean dice +/- std: {trunc(np.round(np.mean(dices_), decimals=4)*100, 2)} ({trunc(np.round(np.std(dices_), decimals=4)*100, 2)})") 
            
            dices_ewc[model.replace('_100_lr-1e-4-with-adapter-lr-1e-3', '').replace('sam_adapter_', '')][datas.split('_')[0].replace('Task', '')]['mean_dice_moved'] = np.mean(dices_)
            dices_ewc[model.replace('_100_lr-1e-4-with-adapter-lr-1e-3', '').replace('sam_adapter_', '')][datas.split('_')[0].replace('Task', '')]['mean_std_moved'] = np.std(dices_)
            dices_ewc[model.replace('_100_lr-1e-4-with-adapter-lr-1e-3', '').replace('sam_adapter_', '')][datas.split('_')[0].replace('Task', '')]['mean_dice_std_moved'] = str(np.round(np.mean(dices_), 4)) + '(' + str(np.round(np.std(dices_), 4)) +')'
        print()

In [ ]:
# RWalk
models = [x for x in os.listdir(rwalk_exps) if 'sam_adapter' in x and 'joint' not in x]

models.sort()
dices_rwalk = dict()
for model in models:
    dices_rwalk[model.replace('_100_lr-1e-4-with-adapter-lr-1e-3', '').replace('sam_adapter_', '')] = dict()
    preds_ = [x for x in os.listdir(os.path.join(rwalk_exps, model)) if 'inference' in x]
    for pred in preds_:
        preds = os.path.join(rwalk_exps, model, pred)
        print(f"Looking in {preds}..")
        # -- Load the data -- #
        try:
            ds = [x for x in os.listdir(preds) if 'csv' not in x]
            ds.sort()
            print(f'Dice for model {model}:')
        except:
            continue
        
        for datas in ds:
            dices_ = list()
            dices_rwalk[model.replace('_100_lr-1e-4-with-adapter-lr-1e-3', '').replace('sam_adapter_', '')][datas.split('_')[0].replace('Task', '')] = dict()
            cases = [x for x in os.listdir(os.path.join(preds, datas)) if 'DS_' not in x]
            for case in cases:
                dices_rwalk[model.replace('_100_lr-1e-4-with-adapter-lr-1e-3', '').replace('sam_adapter_', '')][datas.split('_')[0].replace('Task', '')][case] = dict()
                gt = np.int32(sitk.GetArrayFromImage(sitk.ReadImage(os.path.join(preds, datas, case, 'seg_gt.nii.gz'))))
                y_p = sitk.GetArrayFromImage(sitk.ReadImage(os.path.join(preds, datas, case, 'pred_seg.nii.gz')))
                y_p_t = torch.from_numpy(y_p).to(0).unsqueeze(0).unsqueeze(0).float()#.permute(0, 4, 1, 2, 3)
                gt_t = torch.from_numpy(gt).to(0).unsqueeze(0).unsqueeze(0).float()#.permute(0, 4, 1, 2, 3)
                dice = monai.metrics.DiceMetric(include_background=False, ignore_empty=False)(y_p_t, gt_t).cpu().item()
                dices_rwalk[model.replace('_100_lr-1e-4-with-adapter-lr-1e-3', '').replace('sam_adapter_', '')][datas.split('_')[0].replace('Task', '')][case]['pred'] = np.round(dice, 4)
                dices_.append(dice)

            print(f"{datas}: Mean dice +/- std: {trunc(np.round(np.mean(dices_), decimals=4)*100, 2)} ({trunc(np.round(np.std(dices_), decimals=4)*100, 2)})") 
            
            dices_rwalk[model.replace('_100_lr-1e-4-with-adapter-lr-1e-3', '').replace('sam_adapter_', '')][datas.split('_')[0].replace('Task', '')]['mean_dice_moved'] = np.mean(dices_)
            dices_rwalk[model.replace('_100_lr-1e-4-with-adapter-lr-1e-3', '').replace('sam_adapter_', '')][datas.split('_')[0].replace('Task', '')]['mean_std_moved'] = np.std(dices_)
            dices_rwalk[model.replace('_100_lr-1e-4-with-adapter-lr-1e-3', '').replace('sam_adapter_', '')][datas.split('_')[0].replace('Task', '')]['mean_dice_std_moved'] = str(np.round(np.mean(dices_), 4)) + '(' + str(np.round(np.std(dices_), 4)) +')'
        print()

In [ ]:
data = {'SAM': dices_s, 'EWC': dices_ewc, 'RWalk': dices_rwalk, 'Rehearsal': dices_reh
       }

#### Forgetting, Positive Backward Transfer, Remembering and Forward Transfer

In [ ]:
tasks = ['99', '98', '97']
methods = ['SAM']#, 'EWC', 'RWalk', 'Rehearsal']
B_T, F_T, F_NEG, F_POS, B_TP, FOR, REM, Dice, Dice_F, Dice_L, STD = dict(), dict(), dict(), dict(), dict(), dict(), dict(), dict(), dict(), dict(), dict()
Dice_2 = dict()

all_t_j = '_'.join(tasks)
for m in methods:
    B_T[m], F_T[m], F_NEG[m], F_POS[m], B_TP[m], FOR[m], REM[m], Dice[m], Dice_F[m], Dice_L[m], STD[m] = dict(), dict(), dict(), dict(), dict(), dict(), dict(), dict(), dict(), dict(), dict()
    Dice_2[m] = dict()
    t_list = list()
    for t in tasks:
        t_list.append(t)
        try:
            Dice_t_all = (data[m][all_t_j][t]['mean_dice_moved'],
                            data[m][all_t_j][t]['mean_std_moved'])
            
            Dice_t = (data[m]['_'.join(t_list)][t]['mean_dice_moved'],
                        data[m]['_'.join(t_list)][t]['mean_std_moved'])
            
            if len(t_list) > 1:
                Dice_t_prev = (data[m]['_'.join(t_list[:-1])][t]['mean_dice_moved'],
                                data[m]['_'.join(t_list[:-1])][t]['mean_std_moved'])
                F_T[m][t] = Dice_t_prev[0] - data[m][t][t]['mean_dice_moved']
                F_NEG[m][t] = abs(min(F_T[m][t], 0))
                F_POS[m][t] = 1 - abs(min(F_T[m][t], 0))
            
            if '_'.join(t_list) == all_t_j: # Only add this once
                Dice_F[m][t] = data[m][all_t_j][tasks[0]]['mean_dice_moved']
                Dice_2[m][t] = data[m][all_t_j][tasks[1]]['mean_dice_moved']
                Dice_L[m][t] = data[m][all_t_j][tasks[2]]['mean_dice_moved']
            
            Dice[m][t] = data[m][all_t_j][t]['mean_dice_moved']

            if '_'.join(t_list) != all_t_j:
                B_T[m][t] = Dice_t_all[0] - Dice_t[0]
                B_TP[m][t] = max(B_T[m][t], 0)
                FOR[m][t] = abs(min(B_T[m][t], 0))
                REM[m][t] = 1 - abs(min(B_T[m][t], 0))
        except Exception as e:
            print(e)
            pass

In [ ]:
print('BWT, FWT [%]')
for m in methods:
    print(m)
    print(np.mean(np.asarray(list(B_T[m].values()))*100), '+\-', np.std(np.asarray(list(B_T[m].values()))*100))
    # print(np.mean(np.asarray(list(REM[m].values()))*100), '+\-', np.std(np.asarray(list(REM[m].values()))*100))
    print(np.mean(np.asarray(list(F_T[m].values()))*100), '+\-', np.std(np.asarray(list(F_T[m].values()))*100))
    # print(np.mean(np.asarray(list(Dice[m].values()))*100), '+\-', np.std(np.asarray(list(Dice[m].values()))*100))

In [ ]:
print('Mean Dice [%]')
for m in methods:
    print(m)
    print(np.mean(list(Dice[m].values()))*100)

In [ ]:
print('Mean Forgetting [%]')
for m in methods:
    print(m)
    print(np.mean(list(FOR[m].values()))*100)

#### Spider Plots

In [ ]:
plt.figure(figsize=(13, 6))

df = pd.DataFrame({
'methods': methods,
'Dice Mean': [np.mean(list(Dice[m].values()))*100 for m in methods],
'100 + BWT': [100 + np.mean(list(B_T[m].values()))*100 for m in methods],
'Dice Last': [np.mean(list(Dice_L[m].values()))*100 for m in methods],
'Dice First': [np.mean(list(Dice_F[m].values()))*100 for m in methods],
'100 + FWT': [100 + np.mean(list(F_T[m].values()))*100 for m in methods]
}) 
# ------- PART 1: Define a function that do a plot for one line of the dataset!
 
def make_spider(row, title, color, h_=None):

    # number of variable
    categories=list(df)[1:]
    N = len(categories)

    # What will be the angle of each axis in the plot? (we divide the plot / number of variable)
    angles = [n / float(N) * 2 * pi for n in range(N)]
    angles += angles[:1]

    # Initialise the spider plot
    ax = plt.subplot(3, 3, row+1, polar=True)

    # If you want the first axis to be on top:
    ax.set_theta_offset(pi / 2)
    ax.set_theta_direction(-1)
    # ax.xaxis.set_tick_params(pad=35)    # Add padding to labels

    # Draw one axe per variable + add labels yet
    plt.xticks(angles[:-1], categories, color='black', size=15)
    
    for i, tick in enumerate(ax.get_xaxis().get_major_ticks()):
        if i == 1 or i == 4:
            tick.set_pad(35)
        else:
            tick.set_pad(15)

    # Draw ylabels
    # ax.set_rlabel_position(0)
    plt.yticks(range(0, 101, 20), [str(x) for x in range(0, 101, 20)], color="grey", size=9)
    plt.ylim(0,100)

    # Ind1
    values=df.loc[row].drop('methods').values.flatten().tolist()
    values += values[:1]
    ax.plot(angles, values, color='grey', alpha=0.8, linewidth=2, linestyle='solid')
    if h_ is not None:
        ax.fill(angles, values, color=color, alpha=h_)
    else:
        ax.fill(angles, values, color=color)
            
    # Add a title
    plt.title(title)

In [ ]:
# ------- PART 2: Apply the function to all individuals
colors = {'SAM': '#fae5d6', 'Rehearsal': '#26a671', 'RWalk':'#fec10b', 'EWC':'#9e1e83'}

# Loop to plot
for row in range(0, len(df.index)):

    # Plot one after the other to save as pdf for better plotting
    my_dpi=96
    plt.figure(figsize=(13, 13), dpi=my_dpi)
    plt.subplots_adjust(right=1.1, top=1.1)
    plt.tight_layout()

    make_spider(row=row, title=df['methods'][row], color=colors[methods[row]], h_=0.2 if methods[row] not in ['U-Net', 'Atlas Replay'] else None)
    # make_spider( row=row, title=df['method'][row], color=my_palette(row))
    title = df['methods'][row]
    # plt.savefig(f'/home/aranem_locale/Desktop/MICCAI_2023/MICCAI_2023_evaluation/plots/spiders_{title}.pdf', bbox_inches='tight')

In [ ]:
plt.figure(figsize=(13, 6))

df = pd.DataFrame({
'methods': methods,
'$\mathcal{T}_{1}$': [np.mean(list(Dice_F[m].values()))*100 for m in methods],
'$\mathcal{T}_{2}$': [np.mean(list(Dice_2[m].values()))*100 for m in methods],
'$\mathcal{T}_{3}$': [np.mean(list(Dice_L[m].values()))*100 for m in methods],
}) 
# ------- PART 1: Define a function that do a plot for one line of the dataset!
 
def make_spider(row, title, color, h_=None):

    # number of variable
    categories=list(df)[1:]
    N = len(categories)

    # What will be the angle of each axis in the plot? (we divide the plot / number of variable)
    angles = [n / float(N) * 2 * pi for n in range(N)]
    angles += angles[:1]

    # Initialise the spider plot
    ax = plt.subplot(3, 3, row+1, polar=True)

    # If you want the first axis to be on top:
    ax.set_theta_offset(pi / 2)
    ax.set_theta_direction(-1)
    ax.xaxis.set_tick_params(pad=15)    # Add padding to labels

    # Draw one axe per variable + add labels yet
    # plt.xticks(angles[:-1], categories, color='black', size=15)
    plt.xticks(angles[:-1], categories, color='black', size=31)
    
    # Draw ylabels
    # ax.set_rlabel_position(0)
    # plt.yticks(range(0, 101, 20), [str(x) for x in range(0, 101, 20)], color="grey", size=11)
    plt.yticks(range(0, 101, 20), [str(x) for x in range(0, 101, 20)], color="grey", size=15)
    plt.ylim(0,100)

    # Ind1
    values=df.loc[row].drop('methods').values.flatten().tolist()
    values += values[:1]
    ax.plot(angles, values, color='grey', alpha=0.8, linewidth=2, linestyle='solid')
    if h_ is not None:
        ax.fill(angles, values, color=color, alpha=h_)
    else:
        ax.fill(angles, values, color=color)
            
    # Add a title
    plt.title(title)

In [ ]:
# ------- PART 2: Apply the function to all individuals
colors = {'SAM': '#fae5d6', 'Rehearsal': '#26a671', 'RWalk':'#fec10b', 'EWC':'#9e1e83'}

# Loop to plot
for row in range(0, len(df.index)):

    # Plot one after the other to save as pdf for better plotting
    my_dpi=96
    plt.figure(figsize=(13, 13), dpi=my_dpi)
    plt.subplots_adjust(right=1.1, top=1.1)
    plt.tight_layout()

    make_spider(row=row, title=df['methods'][row], color=colors[methods[row]], h_=0.2 if methods[row] not in ['U-Net', 'Atlas Replay'] else None)
    # make_spider( row=row, title=df['method'][row], color=my_palette(row))
    title = df['methods'][row]
    # plt.savefig(f'/home/aranem_locale/Desktop/MICCAI_2023/MICCAI_2023_evaluation/plots/spiders_{title}.pdf', bbox_inches='tight')